<a href="https://colab.research.google.com/github/JordanDCunha/R-for-Data-Science-2e-/blob/main/Chapter_21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 21.1 Introduction
A huge amount of data lives in **databases**, so knowing how to access them directly is essential. Relying on someone else to export `.csv` snapshots quickly becomes inefficient—every change requires another manual step. Instead, you want to query databases **on demand**, pulling exactly the data you need, when you need it.

In this chapter, you’ll learn the basics of working with databases from R using the **DBI** package. DBI provides the low-level tools to connect to databases and execute queries written in **SQL** (Structured Query Language), the standard language used to interact with databases.

Rather than starting with raw SQL, the chapter introduces **dbplyr**, which translates familiar `dplyr` code into SQL behind the scenes. This lets you:
- Work with databases using tidyverse-style syntax
- Learn key SQL concepts gradually
- Understand how common data manipulation verbs map to SQL operations

By the end of the chapter, you won’t be a SQL expert—but you’ll recognize the most important SQL components and understand how they fit together.

## 21.1.1 Prerequisites
This chapter uses:
- **DBI** for database connections and query execution
- **dbplyr** to translate `dplyr` code into SQL
- **tidyverse** for data manipulation workflows


In [ ]:
library(DBI)
library(dbplyr)
library(tidyverse)




# 21.2 Database basics
At a high level, a **database** can be thought of as a collection of data frames, called **tables**. Like data frames, tables consist of named columns where each column contains values of a single type. However, there are some important differences:

### Data frames vs database tables
- **Storage**: Database tables live on disk and can be arbitrarily large, while data frames live in memory and are limited by available RAM.
- **Indexes**: Database tables usually have **indexes**, which allow fast lookup of specific rows (similar to a book index). Data frames don’t have indexes (though `data.table` does, which helps explain its speed).
- **Optimization**: Traditional databases are optimized for **data collection**, not analysis. They are typically **row-oriented** (stored row by row), whereas R works column-by-column. Newer **column-oriented** databases greatly improve analytical performance.

### Types of database management systems (DBMSs)
Databases are managed by DBMSs, which generally fall into three categories:

- **Client–server DBMSs**  
  Run on a central server and are accessed by multiple clients. Ideal for shared, organizational data.  
  Examples: PostgreSQL, MariaDB, SQL Server, Oracle.

- **Cloud DBMSs**  
  Similar to client–server systems but hosted in the cloud, allowing elastic scaling and massive datasets.  
  Examples: Snowflake, Amazon Redshift, Google BigQuery.

- **In-process DBMSs**  
  Run entirely on your local machine. Best when you’re the primary user working with large datasets.  
  Examples: SQLite, duckdb.


# 21.3 Connecting to a database

To connect to a database from R, you typically use **two kinds of packages**:

### DBI (Database Interface)
- **DBI** provides a common set of functions for connecting to databases, sending queries, and retrieving results.
- You will *always* use DBI, regardless of the database.

### A DBMS-specific package
- Each database management system (DBMS) has its own package that translates DBI’s generic commands into database-specific instructions.
- Examples:
  - PostgreSQL → `RPostgres`
  - MySQL / MariaDB → `RMariaDB`
- If no dedicated package exists, you can usually fall back on **`odbc`**, which works with many databases but requires extra setup (drivers + configuration).

### Creating a connection
Connections are created with `DBI::dbConnect()`.  
- The **first argument** specifies the DBMS.
- The remaining arguments describe how to connect (host, port, credentials, etc.).
- Connection details vary by database, so some trial and error (and help from a DBA or teammates) is normal.

---

## 21.3.1 In this book: duckdb
Instead of setting up a server or cloud database, this book uses **duckdb**, an **in-process DBMS** that runs entirely inside R.

Why duckdb?
- Extremely easy to set up
- Very fast and designed for data analysis
- Uses DBI, so everything you learn transfers to other databases

By default, duckdb creates a **temporary database** that disappears when R closes.  
For real projects, you can create a **persistent database** by supplying a directory.

---

## 21.3.2 Loading data
Once connected, you need to load data into the database.  
`DBI::dbWriteTable()` creates a table from a data frame.

For real-world duckdb usage, faster alternatives include:
- `duckdb_read_csv()`
- `duckdb_register_arrow()`

(these avoid loading data fully into R first).

---

## 21.3.3 DBI basics
Useful DBI functions include:
- `dbListTables()` — list tables in the database
- `dbReadTable()` — read an entire table into R
- `dbGetQuery()` — run raw SQL and return the result

If SQL is new to you, don’t stress: you’ll learn it gradually. For now, it’s enough to recognize that SQL queries specify **columns to select** and **rows to keep**.


In [ ]:
# Load required packages
library(DBI)
library(duckdb)
library(tidyverse)

# Connect to a temporary duckdb database
con <- DBI::dbConnect(duckdb::duckdb())

# Load example datasets into the database
dbWriteTable(con, "mpg", ggplot2::mpg)
dbWriteTable(con, "diamonds", ggplot2::diamonds)

# List tables in the database
dbListTables(con)

# Read a table back into R
con |>
  dbReadTable("diamonds") |>
  as_tibble()

# Run a raw SQL query
sql <- "
  SELECT carat, cut, clarity, color, price
  FROM diamonds
  WHERE price > 15000
"

as_tibble(dbGetQuery(con, sql))


# 21.4 dbplyr basics

Now that we’ve connected to a database and loaded data, we can introduce **dbplyr**.  
dbplyr is a *dplyr backend*, meaning you keep writing familiar **dplyr** code, but instead of running directly in R, your code is **translated into SQL** and executed inside the database.

Other dplyr backends include:
- **dtplyr** → translates to `data.table`
- **multidplyr** → runs code across multiple cores

---

## Working with database tables
To use dbplyr, you first create a reference to a database table using `tbl()`:

- This does **not** load the data into R.
- It creates a *lazy* object that represents a table or query in the database.

In larger systems, tables may live inside **schemas** or **catalogs**, which help organize many tables.  
You can also start from a **custom SQL query** if needed.

---

## Lazy evaluation
dbplyr objects are *lazy*.  
When you apply dplyr verbs like `filter()` or `select()`, **no computation happens immediately**. Instead, dbplyr records the operations and only runs them when results are actually needed.

This is why:
- You often don’t see the number of rows
- Printing is fast, even for huge datasets

---

## Translating dplyr to SQL
You can inspect the SQL generated by your dplyr code using `show_query()`.  
This is a powerful way to *learn SQL by example*: write dplyr, then see how it maps to SQL.

---

## Bringing data back into R
To actually retrieve results from the database, use `collect()`:
- dbplyr generates the SQL
- DBI runs the query
- Results are returned as a tibble in R

**Typical workflow**:
1. Filter, select, and aggregate data in the database (fast, scalable)
2. `collect()` once the dataset is small enough
3. Continue analysis with R-specific tools


In [ ]:
# Load required packages
library(DBI)
library(dbplyr)
library(duckdb)
library(tidyverse)

# Connect to duckdb
con <- DBI::dbConnect(duckdb::duckdb())

# Reference a database table (lazy)
diamonds_db <- tbl(con, "diamonds")

# Build a lazy query using dplyr
big_diamonds_db <- diamonds_db |>
  filter(price > 15000) |>
  select(carat:clarity, price)

# View the generated SQL
big_diamonds_db |>
  show_query()

# Collect results into R
big_diamonds <- big_diamonds_db |>
  collect()

big_diamonds


# 21.5 SQL

This section introduces **SQL through dbplyr**, using your existing dplyr knowledge as a bridge. Rather than learning SQL syntax in isolation, you see how familiar dplyr verbs are **translated into SQL queries** behind the scenes.

We work with the **nycflights13** datasets (`flights`, `planes`) copied into a database, then explore how dplyr pipelines map onto SQL.

---

## SQL structure
SQL queries are built from **clauses**. The most important ones are:

- **SELECT** – choose, create, or rename columns  
- **FROM** – specify the data source  
- **WHERE** – filter rows  
- **GROUP BY** – define groups for aggregation  
- **ORDER BY** – sort rows  

Every query must include `SELECT` and `FROM`.  
Although SQL is written in the order `SELECT → FROM → WHERE → GROUP BY → ORDER BY`, it is *evaluated* in a different order internally.

---

## dplyr ↔ SQL mappings

### SELECT
- `select()`, `rename()`, `relocate()` → column selection and aliasing
- `mutate()` → computed expressions inside `SELECT`
- Renaming uses **AS** in SQL
- Reserved words (e.g., `year`, `type`) are quoted automatically

### FROM
- Defines the table or subquery used as input
- Becomes more interesting once joins are introduced

### WHERE
- `filter()` → `WHERE`
- `|` → `OR`, `&` → `AND`
- `==` → `=`
- `%in%` → `IN`
- SQL uses **NULL** instead of `NA`
- Filters on summarized values generate **HAVING**, not `WHERE`

### GROUP BY
- `group_by()` → `GROUP BY`
- `summarize()` → aggregation expressions in `SELECT`
- SQL drops NULLs automatically in summaries (dbplyr warns you)

### ORDER BY
- `arrange()` → `ORDER BY`
- `desc()` → `DESC`

---

## Subqueries
Sometimes dbplyr must generate **subqueries** to match SQL’s evaluation rules.  
This happens when:
- You reference a variable created earlier in the same pipeline
- You filter on a newly created column

Subqueries appear as nested `SELECT` statements in the `FROM` clause.

---

## Joins
dplyr joins map almost directly to SQL joins:

- `left_join()` → `LEFT JOIN`
- `inner_join()` → `INNER JOIN`
- `right_join()` → `RIGHT JOIN`
- `full_join()` → `FULL JOIN`

SQL joins live inside the `FROM` clause and use **ON** to define key relationships.

---

## Beyond the basics
dbplyr also translates:
- `distinct()`
- `slice_*()`
- set operations like `intersect()`
- selected tidyr verbs (e.g., `pivot_longer()`, `pivot_wider()`)

Because SQL dialects vary across databases, dbplyr adapts its translations automatically—imperfectly, but increasingly well.

---

## Key idea
Use **databases for filtering, grouping, and joining**, then `collect()` once the data is small enough for in-memory R analysis. This gives you scalability without giving up dplyr.


In [ ]:
# Load packages
library(DBI)
library(dbplyr)
library(tidyverse)
library(nycflights13)

# Copy nycflights13 tables into the database
dbplyr::copy_nycflights13(con)

# Reference tables lazily
flights <- tbl(con, "flights")
planes  <- tbl(con, "planes")

# Example: filtering and ordering
flights |>
  filter(dest %in% c("IAH", "HOU")) |>
  arrange(desc(dep_delay)) |>
  show_query()

# Example: grouping and summarizing
flights |>
  group_by(dest) |>
  summarize(avg_delay = mean(arr_delay, na.rm = TRUE)) |>
  show_query()

# Example: join
flights |>
  left_join(planes |> rename(year_built = year), join_by(tailnum)) |>
  show_query()


# 21.6 Function translations

This section zooms in on how **individual R functions** (used inside `summarize()` and `mutate()`) are translated by **dbplyr** into SQL expressions.

## Aggregation vs window functions
- In `summarize()`, functions like `mean()` or `median()` become **SQL aggregation functions** (e.g., `AVG()`, `MEDIAN()`).
- In `mutate()`, the same functions become **window functions**, using `OVER (...)`, because results must be computed per row rather than collapsing rows.

## Window functions
- Grouping variables move from `GROUP BY` into `PARTITION BY` inside `OVER`.
- Ordering must be explicit with `arrange()` because SQL tables have no inherent row order.
- Functions like `lead()` and `lag()` are classic window functions and require both partitioning and ordering.

## Conditional logic
- `if_else()` and `case_when()` translate to SQL’s **CASE WHEN** syntax.
- Functions without direct SQL equivalents (e.g., `cut()`) are also implemented using `CASE WHEN`.

## Key takeaway
dbplyr translates many common R functions into SQL—sometimes simply, sometimes with complex expressions. When translations are unavailable, computation must happen after `collect()`. For most day-to-day analysis, dbplyr covers the functions you’ll use most often.
